# Basic MlFlow Test

## Experiment Tracking

In [2]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

# Enable MLflow's automatic experiment tracking for scikit-learn
mlflow.sklearn.autolog()

# Load the training dataset
db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
# MLflow triggers logging automatically upon model fitting
rf.fit(X_train, y_train)

2025/01/19 15:07:56 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.2, but the installed version is 1.6.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.


2025/01/19 15:07:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '125e32eb708f4c8681f42c11c80b9729', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


RandomForestRegressor(max_depth=6, max_features=3)

## Evaluating Models

In [1]:
import mlflow
import pandas as pd

# Evaluation set contains (1) input question (2) model outputs (3) ground truth
df = pd.DataFrame(
    {
        "inputs": ["What is MLflow?", "What is Spark?"],
        "outputs": [
            "MLflow is an innovative fully self-driving airship powered by AI.",
            "Sparks is an American pop and rock duo formed in Los Angeles.",
        ],
        "ground_truth": [
            "MLflow is an open-source platform for managing the end-to-end machine learning (ML) "
            "lifecycle.",
            "Apache Spark is an open-source, distributed computing system designed for big data "
            "processing and analytics.",
        ],
    }
)
eval_dataset = mlflow.data.from_pandas(
    df, predictions="outputs", targets="ground_truth"
)

# Start an MLflow Run to record the evaluation results to
print("Starting MLflow run...")
with mlflow.start_run(run_name="evaluate_qa"):
    # Run automatic evaluation with a set of built-in metrics for question-answering models
    print("Evaluating model...")
    results = mlflow.evaluate(
        data=eval_dataset,
        model_type="question-answering",
    )

print(results.tables["eval_results_table"])

Starting MLflow run...
Evaluating model...


2025/01/19 18:25:31 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/01/19 18:25:34 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/01/19 18:25:34 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/01/19 18:25:34 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/01/19 18:25:34 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
/opt/homebrew/Caskroom/mambaforge/base/envs/mlflow-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthe

            inputs                                       ground_truth  \
0  What is MLflow?  MLflow is an open-source platform for managing...   
1   What is Spark?  Apache Spark is an open-source, distributed co...   

                                             outputs  token_count  \
0  MLflow is an innovative fully self-driving air...           14   
1  Sparks is an American pop and rock duo formed ...           14   

   flesch_kincaid_grade_level/v1/score  ari_grade_level/v1/score  
0                                  9.6                       9.9  
1                                  4.4                       4.2  


## Observability

In [ ]:
import mlflow
from openai import OpenAI

# Enable tracing for OpenAI
mlflow.openai.autolog()

# Query OpenAI LLM normally
response = OpenAI().chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hi!"}],
    temperature=0.1,
)